<a href="https://colab.research.google.com/github/MateoPeri/steel-scrap-predictor/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import glob

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from fbprophet import Prophet
import matplotlib.pyplot as plt
import altair as alt
plt.style.use('seaborn-poster')

In [4]:
!rm -rf '/content/data'
!mkdir data
PATH = '/content/data/'
!wget -P {PATH} https://raw.githubusercontent.com/MateoPeri/steel-scrap-predictor/master/data/raw.zip
!unzip {PATH}raw.zip

--2020-06-05 14:42:06--  https://raw.githubusercontent.com/MateoPeri/steel-scrap-predictor/master/data/raw.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 240524 (235K) [application/zip]
Saving to: ‘/content/data/raw.zip’

raw.zip             100%[===================>] 234.89K  --.-KB/s    in 0.05s   

2020-06-05 14:42:07 (4.53 MB/s) - ‘/content/data/raw.zip’ saved [240524/240524]

Archive:  /content/data/raw.zip
   creating: raw/
  inflating: raw/HC Closing Prices.xlsx  
  inflating: raw/HC Final Settlement prices.xlsx  
  inflating: raw/HU Closing Prices.xlsx  
  inflating: raw/HU Final Settlement prices.xlsx  
  inflating: raw/SC Closing Prices.xlsx  
  inflating: raw/SC Final Settlement prices.xlsx  
  inflating: raw/SR Closing Prices.xlsx  
  inflating: 

In [0]:
def st_to_mt(st):
  return st/1.10231131

In [47]:
pattern = 'raw/*Closing Prices.xlsx'
excels = glob.glob(pattern)

frames = []

for xcel in excels:
  df = pd.read_excel(xcel, parse_dates=True, sep=';', escapechar='\\', encoding='ISO-8859-1', decimal=',', thousands='.')
  name = xcel[4:6]
  if (name == 'HU'):
    # Seleccionar todas las columnas menos USD/st
    cols = [c for c in df.columns if c != 'USD/st (Short Ton)']
    # y aplicarles la funcion st_to_mt para pasar a toneladas métricas
    df[cols] = df[cols].apply(st_to_mt, axis=1)
    # Cambiamos el nombre a la columna
    df.rename({'USD/st (Short Ton)': 'USD/mt'}, axis=1, inplace=True)
  # le añadimos un nombre para saber qué metal es
  df['Metal'] = name
  frames.append(df)

df = pd.concat(frames, ignore_index=True)

df.head()

,USD/mt,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12,M13,M14,M15,Metal
0,2015-11-23,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,SC
1,2015-11-24,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,195.0,SC
2,2015-11-25,195.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,SC
3,2015-11-26,192.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,SC
4,2015-11-27,192.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,193.0,SC


In [57]:
print(df.shape)
# Contamos los valores nulos del dataset -> 0
df.info()

(2835, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2835 entries, 0 to 2834
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   USD/mt  2835 non-null   datetime64[ns]
 1   M1      2835 non-null   float64       
 2   M2      2835 non-null   float64       
 3   M3      2835 non-null   float64       
 4   M4      2835 non-null   float64       
 5   M5      2835 non-null   float64       
 6   M6      2835 non-null   float64       
 7   M7      2835 non-null   float64       
 8   M8      2835 non-null   float64       
 9   M9      2835 non-null   float64       
 10  M10     2835 non-null   float64       
 11  M11     2835 non-null   float64       
 12  M12     2835 non-null   float64       
 13  M13     2835 non-null   float64       
 14  M14     2835 non-null   float64       
 15  M15     2835 non-null   float64       
 16  Metal   2835 non-null   object        
dtypes: datetime64[ns](1), float64(15), object